## The theory for these notebooks/codes


Here we compute a 1D version of PIC. We mostly follows the paper: *SHARP: A SPATIALLY HIGHER-ORDER, RELATIVISTIC PARTICLE-IN-CELL CODE*, arXiv:1702.04732v2

**But we change several things:**

1. The integration is done with RK4.
2. The electric field is integrated along the simulation and the constraint is checked afterwards.
   

**Initial data sets** We tried several different initial data sets:

1. Thermal distributions.
2. Counter streaming distributions.
3. Landau: it corresponds to particles in a thermal bath with a given temperature and a spatial distribution given by a cosine function over a background.

The distribution is (relativistic version, we also had some non-relativistic code and the corresponding distributions.):

$$
f(x,v) = \frac{N}{L} \frac{1}{\sqrt{2\pi\theta}} e^{\frac{1 - \sqrt{1 + p^2}}{\theta}}(1 + \alpha cos(kx))
$$

We shall use $\theta = 0.001$, $\alpha = 0.001$ (some with $\alpha = 0.1$ for fun).

The normalization factors are not important because we then normalize the momentums of the distribution.




We have some extra quantities that are proper of the plasma:

The plasma frequency and the Debye temperature:

The plasma frequency is adjusted so that it is unit in with respect to the time units 
taken, space it taken so that the speed of light is one. Taking $\epsilon_0 = 1$,
$$
\omega_p^2 = \frac{m_e}{e^2 N_e} = 1
$$

This implies (since we take $e=1$, $q=1$) that the background density is $N_e=1$.
Thus, we need to rescale densities and currents dividing by $n_0 = N/L$

Similarly we rescale the electric field with $\sqrt{N_e m_e}$, and space so that $c=1$.

In the paper they say the temperature is set to $\theta_D = dx^2$. 
This I don't understand yet. I will call it $\theta_h = dx^2$. 
Correspondingly there is a wavelength, $\lambda_D= dx$, which I will call $\lambda_h$.

There is also another temperature which seems to be relevant, that is the Poisson temperature, which is relevant to see the degree of Poisson noice of the superparticles. 
This is given by,

$$
\theta_P  \approx \theta_h \frac{J^2}{12N} \approx \frac{L^2}{J^2}\frac{J^2}{12N} = \frac{L^2}{12N}
$$

Below this value, temperatures are not well defined.


Some definitions $\hat{k} = k / k_D := k * \sqrt{\theta}$ Or $k := \hat{k} / \sqrt{\theta}$ 

In these simulations, $\theta = 10^{-3}$, that is, $\hat{k} =\sqrt{\theta} = 0.03162$

Thus to $\hat{k} = 0.01$ corresponds a $\lambda = 2\pi * \sqrt{\theta} / \hat{k} = 19.869$

For $\hat{k} = 0.40$ corresponds a $\lambda = 0.497$

For the first case an $L = 2 \lambda = 39.738$ is used and $J = 1761*2 = 3522$, $dx = L/J = 0.0112$

For the second $L=15*\lambda = 7.455$ is used and $J = 62 * 15 = 930$, $dx = 0.0080$

### The equations:

For the particles we have:

\begin{array}{cc}
    \frac{dp}{dt} &= e E(x) \\
    \frac{dx}{dt} &= v = \frac{\frac{p}{m}}{\sqrt{1+\frac{p}{m}^2}}
\end{array}


For the electric field we have,

$$
\frac{dE}{dt} = -eS
$$

Plus the constraint:

$$
\partial_x E = n
$$

Where $(n,S)$ are the density and current (properly normalized).


We recall that:

$$
n(x) = \frac{1}{m_e}\int_p f(x,p) dp 
$$

and that,

$$
S(x) = \frac{1}{m_e}\int_p f(x,p) v(p) dp 
$$


### Landau frequencies

For small $\hat{k}$ we can use the approximations: 

$$
\omega_r = 1 + 3/2 \hat{k}^2, \;\;\;\;\;\;\;\; \omega_i = -\sqrt{\frac{\pi}{8}} \frac{1}{\hat{k}^3} e^{-\frac{1}{\hat{k}^2} - \frac{3}{2}} 
$$

for $\hat{k}=0.01$ we get,

$$
\omega = 1.00015 + i*0.0
$$

for $\hat{k} = 0.4$ we get, (this is from the paper for this value of $\hat{k}$ is too big for the approximation to be valid).

$$
\omega = 1.28506 - i*0.06613
$$

This corresponds to $k= 0.3162$ and $k= 12.649$ respecively for $\theta=0.001$.  

For $\theta=0.1$ the $\hat{k}$'s are 10 times bigger (keeping the same k's). Thus, for the undamped case, $k=0.3162$, $\hat{k}=0.1$
and

$$
\omega \approx 1.015 + i*0.0
$$



In [6]:
function approx_omega(k)
    return 1 + 3*k^2/2 - im*(sqrt(π/8)/k^3*exp(-1/2/k^2 - 3/2))
end

@show approx_omega(0.01)
@show approx_omega(0.4); # this value is not correct because the approximation does not hold.

approx_omega(0.01) = 1.00015 - 0.0im
approx_omega(0.4) = 1.24 - 0.09599265198249836im


In [5]:
k = 0.01 / sqrt(0.001) * sqrt(0.1)

0.1

In [3]:
2π/40*2

0.3141592653589793

### Numerical scheme:

In the numerical approximation for the evolution we compute the electric field at fixed grid points, $\{x_j = dx*(j-1)\}, j=1\ldots J$ $dx = L/J$ and then interpolate to the particle point to get the force. For this we use the scheme, using the weight functions from the paper above and their averaged method. For computing the time derivative of the electric field, we compute the current using the shape/weight functions. 

The initial data for the particles is choosen random acording to the distribution functions considered. The initial electric field is computed solving the constraints, having previously calculated the initial density.
